In [44]:
import torch

num_class = 4
input_size = 4
hidden_size = 8
embedding_size = 10
num_layers = 2
batch_size = 1
seq_len = 5


In [45]:
idx2char = ['e', 'h', 'l', 'o']
x_data = [1, 0, 2, 2, 3]
y_data = [3, 1, 2, 3, 2]
one_hot_lookup = [[1, 0, 0, 0],
                    [0, 1, 0, 0],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]]
x_one_hot = [one_hot_lookup[x] for x in x_data]
inputs = torch.LongTensor(x_data).view(1,-1)
labels = torch.LongTensor(y_data)

In [46]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.emb = torch.nn.Embedding(input_size, embedding_size)
        self.rnn = torch.nn.RNN(input_size=embedding_size,
                                hidden_size=hidden_size,
                                num_layers=num_layers,
                                batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, num_class)

    def forward(self, x):
        # 初始化 hidden，维度应为 (num_layers, batch_size, hidden_size)
        hidden = torch.zeros(num_layers, batch_size, hidden_size)
        x = self.emb(x)  # (batch, seq_len, embedding_size)
        x, _ = self.rnn(x, hidden)  # 传入 RNN
        x = self.fc(x)  # (batch, seq_len, num_class)
        return x.view(-1, num_class)


net = Model()

In [47]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)

In [48]:
for epoch in range(15):
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    _, idx = outputs.max(dim=1)
    idx = idx.data.numpy()
    print('Predicted: ', ''.join([idx2char[x] for x in idx]), end='')
    print(', Epoch [%d/15] loss = %.3f' % (epoch + 1, loss.item()))


Predicted:  ooooo, Epoch [1/15] loss = 1.508
Predicted:  ooooo, Epoch [2/15] loss = 1.102
Predicted:  ohlol, Epoch [3/15] loss = 0.715
Predicted:  lhlol, Epoch [4/15] loss = 0.509
Predicted:  ohlol, Epoch [5/15] loss = 0.345
Predicted:  ohlol, Epoch [6/15] loss = 0.210
Predicted:  ohlol, Epoch [7/15] loss = 0.110
Predicted:  ohlol, Epoch [8/15] loss = 0.054
Predicted:  ohlol, Epoch [9/15] loss = 0.029
Predicted:  ohlol, Epoch [10/15] loss = 0.017
Predicted:  ohlol, Epoch [11/15] loss = 0.011
Predicted:  ohlol, Epoch [12/15] loss = 0.007
Predicted:  ohlol, Epoch [13/15] loss = 0.005
Predicted:  ohlol, Epoch [14/15] loss = 0.003
Predicted:  ohlol, Epoch [15/15] loss = 0.002
